# Increase throughput for Llama2-7b Model using Batching techniques on SageMaker LMI v5

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

---

In this notebook, we explore how to use different batching techniques to increase throughput for LLama2-7b large language model on SageMaker using LMI v5 container. We use DJLServing as the model serving solution in this example that is bundled in the LMI container. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to this link (https://docs.djl.ai/docs/serving/index.html).

Batching helps to increase throughput for Generative AI inferencing by combining requests and sending them together to the LLM as a batch. We explore three batching techniques i.e. Dynamic Batching, Continuous Batching and Paged Attention Batching in this notebook and demonstrate the achieved throughput gains.

We utilize SageMaker LMI v5 container which provides rolling batch capability for Continuous Batching along with Paged Attention. In this notebook, we deploy https://huggingface.co/TheBloke/Llama-2-7B-fp16 model across GPUs on a ml.g5.12xlarge instance.

### Import required libraries and establish session using SageMaker SDK

In [ ]:
!pip install sagemaker boto3 huggingface_hub --upgrade --quiet

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

In [ ]:
model_bucket = sess.default_bucket()  # bucket to house model artifacts
s3_code_prefix = "hf-large-model-djl/meta-llama/Llama-2-7b-fp16/code"  # folder within bucket where code artifact will go

s3_model_prefix = "hf-large-model-djl/meta-llama/Llama-2-7b-fp16/model"  # folder within bucket where model artifact will go
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

### [OPTIONAL] Download the model from Hugging Face and upload the model artifacts on Amazon S3

If you intend to download your copy of the model and upload it to a s3 location in your AWS account, please follow the below steps, else you can skip to the next step.

In [ ]:
"""from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = "TheBloke/Llama-2-7b-fp16"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.txt", "*.model", "*.safetensors", "*.bin", "*.chk", "*.pth"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name, cache_dir=local_model_path, allow_patterns=allow_patterns
)"""

In [ ]:
# upload files from local to S3 location
# pretrained_model_location = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
# print(f"Model uploaded to --- > {pretrained_model_location}")

In [ ]:
# Cleanup locally stored model files post S3 upload
#!rm -rf {model_download_path}

### Define a variable to contain the s3 url of the location that has the model

In [ ]:
# Define a variable to contain the s3 url of the location that has the model. For demo purpose, we use Llama-2-7b-fp16 model artifacts from our S3 bucket
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/llama-2/fp16/7B/"

## Deploy 3 endpoints for benchmarking with settings to enable different Batching techniques

We will deploy 3 different endpoints for benchmarking with different Batching techniques as below:

- Dynamic Batching
- Continuous Batching
- Paged Attention Batching

### 1. Dynamic Batching
#### 1.1 Create serving.properties for Dynamic Batching
This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -

    engine: The engine for DJL to use. In this case, we have set it to Python (Dynamic Batching) or MPI (Continuous and Paged Attention Batching).
    option.model_id: The model ID of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models) or S3 path to the model artifacts. 
    option.tensor_parallel_degree: Set to the number of GPU devices over which Accelerate needs to partition the model. This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.

In [ ]:
!rm -rf code_llama2_7b_fp16
!mkdir -p code_llama2_7b_fp16

In [ ]:
%%writefile code_llama2_7b_fp16/serving.properties
engine = Python
option.entryPoint = djl_python.huggingface
option.tensor_parallel_degree = 2
batch_size = 64
max_batch_delay = 1000
option.model_loading_timeout = 900
option.model_id = {{model_id}}

In [ ]:
# we plug in the appropriate model location into our `serving.properties`
template = jinja_env.from_string(Path("code_llama2_7b_fp16/serving.properties").open().read())
Path("code_llama2_7b_fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_llama2_7b_fp16/serving.properties | cat -n

**Image URI for the DJL container is being used here**

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

**Create the Tarball and then upload to S3 location**

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_7b_fp16

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

#### 1.2 Deploy endpoint for Dynamic Batching

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-7b-fp16-mpi")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 900,
            "ContainerStartupHealthCheckTimeoutInSeconds": 900,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### Wait for endpoint to be In-service. This can take a while, so please be patient

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### 2. Continuous Batching
#### 2.1 Create serving.properties for Continuous Batching

In [ ]:
!rm -rf code_llama2_7b_fp16
!mkdir -p code_llama2_7b_fp16

In [ ]:
%%writefile code_llama2_7b_fp16/serving.properties
engine = MPI
option.entryPoint = djl_python.huggingface
option.rolling_batch = auto
option.max_rolling_batch_size = 64
option.paged_attention = false
option.max_rolling_batch_prefill_tokens = 16080
option.tensor_parallel_degree = 2
option.model_loading_timeout = 900
option.model_id = {{model_id}}

In [ ]:
# we plug in the appropriate model location into our `serving.properties`
template = jinja_env.from_string(Path("code_llama2_7b_fp16/serving.properties").open().read())
Path("code_llama2_7b_fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_llama2_7b_fp16/serving.properties | cat -n

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_7b_fp16

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

#### 2.2 Deploy endpoint for Continuous Batching

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-7b-fp16-mpi")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 900,
            "ContainerStartupHealthCheckTimeoutInSeconds": 900,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### This can take a while, so please be patient

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### 3. Paged Attention Batching
#### 3.1 Create serving.properties for Paged Attention Batching

In [ ]:
!rm -rf code_llama2_7b_fp16
!mkdir -p code_llama2_7b_fp16

In [ ]:
%%writefile code_llama2_7b_fp16/serving.properties
engine = MPI
option.entryPoint = djl_python.huggingface
option.rolling_batch = auto
option.max_rolling_batch_size = 64
option.paged_attention = true
option.max_rolling_batch_prefill_tokens = 16080
option.tensor_parallel_degree = 2
option.model_loading_timeout = 900
option.model_id = {{model_id}}

In [ ]:
# we plug in the appropriate model location into our `serving.properties`
template = jinja_env.from_string(Path("code_llama2_7b_fp16/serving.properties").open().read())
Path("code_llama2_7b_fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_llama2_7b_fp16/serving.properties | cat -n

In [ ]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

In [ ]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_7b_fp16

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

#### 3.2 Deploy endpoint for Paged Attention Batching

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-7b-fp16-mpi")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 900,
            "ContainerStartupHealthCheckTimeoutInSeconds": 900,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

#### This can take a while, so please be patient

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

### Benchmark your model 

This is a generative model, so we pass in a Text as a prompt and the Model will complete the sentence and return the results. We will use awscurl command line tool to try it (the awscutl command line tool requires java)

We pass a multiple prompts as input to the model. This is done by downloading below benchmarking tool and setting up the desired performance test env for running tests.

**Following steps need to be run in a studio terminal or EC2 or Notebook instance**. In below example we use a concurrency of 50 clients sending 100 requests from a g5.12xlarge studio notebook terminal, however if you are looking to run lower/higher concurrency benchmarking then feel free to use a lower/higher compute instance to run the concurrent benchmark tests.

#### Install Java using the shell

In [ ]:
!sudo yum -y update
!sudo yum -y install java wget

#### Download the benchmarking tool

In [ ]:
!wget https://github.com/frankfliu/junkyard/releases/download/v0.3.1/awscurl
!chmod +x awscurl

#### Create prompts to be used with Dynamic Batching

In [ ]:
!mkdir dyn_prompts
!echo '{"inputs":"The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt1.txt
!echo '{"inputs":"Write a program to add two numbers in python","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt2.txt
!echo '{"inputs":"Generate a list of ten titles for my book. The book is about my travels around the world, experiencing different cultures and cuisines, meeting many different personalities and finally settling in a remote village in the himalayas","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > dyn_prompts/dyn_prompt3.txt

#### Set up credentials using env vars or use .aws/credentials file

In [ ]:
!export AWS_ACCESS_KEY_ID=<Update here>
!export AWS_SECRET_ACCESS_KEY=<Update here>
!export AWS_SESSION_TOKEN=<Update here>

## Dynamic Batching Benchmarking

We run the benchmarking tool to get the results for Dynamic Batching. You can change the concurrency through -c and number of requests through -N, however please ensure that you are using an instance with enough compute to run the test and endpoint is deployed on a capable instance to handle the concurrency. Run awscurl -h to get more help on the benchmark tool.

In [ ]:
!EXCLUDE_INPUT_TOKEN=1 TOKENIZER=TheBloke/Llama-2-7b-fp16 ./awscurl -c 50 -N 100 -X POST INSERTDYNAMICENDPOINTURLHERE --connect-timeout 60 -H "Content-Type: application/json" --dataset dyn_prompts -t -n sagemaker

#### After the benchmark tests are completed, we will see the results in below format. Below values are sample and actual results will vary based on env setup.

- Total time: 25073.52 ms.
- Non 200 responses: 0, error rate: 0.00
- Concurrent clients: 2
- Total requests: 4
- TPS: 0.16/s
- Total token: 512
- token/req: 128
- token/s: 20.42/s
- Average Latency: 12359.70 ms.
- P50: 14058.52 ms.
- P90: 14059.23 ms.
- P99: 14059.23 ms.

Please make a note of the actual values for later comparison

## Continuous Batching Benchmarking

We run the benchmarking tool to get the results for Continuous Batching.

In [ ]:
# Set up prompts to be used for Continuous and Paged Attention Batching
!mkdir prompts
!echo '{"inputs":"The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the","parameters":{"min_new_tokens":64, "max_new_tokens":64, "do_sample":true}}' > prompts/prompt1.txt
!echo '{"inputs":"Write a program to add two numbers in python","parameters":{"min_new_tokens":256, "max_new_tokens":256, "do_sample":true}}' > prompts/prompt2.txt
!echo '{"inputs":"Generate a list of ten titles for my book. The book is about my travels around the world, experiencing different cultures and cuisines, meeting many different personalities and finally settling in a remote village in the hills","parameters":{"min_new_tokens":128, "max_new_tokens":128, "do_sample":true}}' > prompts/prompt3.txt

In [ ]:
!TOKENIZER=TheBloke/Llama-2-7b-fp16 ./awscurl -c 2 -N 2 -X POST INSERTCONTINUOUSENDPOINTURLHERE --connect-timeout 60   -H "Content-Type: application/json" --dataset prompts -t -n sagemaker

Please make a note of actual values from the benchmark tests for later comparison

## Benchmarking throughput for Paged Attention Batching

We run the benchmarking tool to get the results for Paged Attention Batching.

In [ ]:
!TOKENIZER=TheBloke/Llama-2-7b-fp16 ./awscurl -c 2 -N 2 -X POST INSERTPAGEDENDPOINTURLHERE --connect-timeout 60   -H "Content-Type: application/json" --dataset prompts -t -n sagemaker

Please make a note of actual values from the benchmark tests for later comparison

## We can now compare the throughput results e.g. Token/s, TPS for different batching techniques to review the throughout gains achieved by using Continuous and Paged Attention Batching over Dynamic Batching.


| Model | Batching strategy       | TPS | Token/s |
|-------|-------------------------|----------|--------------|
|llama2-7b   | Dynamic Batching        |     2.62 |       336.28 |
|llama2-7b       | Continuous Batching     |     6.14 |       849.48 |
|llama2-7b       | PagedAttention Batching |     6.29 |       889.68 |

## Clean Up
Delete the resources (Endpoint, Endpoint config, Model) deployed for the 3 endpoints used in above tests.

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|llama2-7b-batching-throughput|llama2-7b-batching-throughput.ipynb)
